# SENTI-PRED: Pipeline Completo de Analise de Sentimentos
## Autor: Pedro Morato Lahoz
## Data: Outubro 2025

---

### Indice:
1. [Configuracao Inicial](#config)
2. [Analise Exploratoria (EDA)](#eda)
3. [Pre-processamento](#preprocessing)
4. [Modelagem](#modeling)
5. [Avaliacao](#evaluation)
6. [Deploy com Docker](#deploy)

---
## 1. Configuracao Inicial <a id='config'></a>

In [ ]:
# Importacoes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import sys
import joblib
import warnings
warnings.filterwarnings('ignore')

# ML imports
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    roc_curve, auc, precision_recall_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# Configuracoes visuais
plt.style.use('ggplot')
sns.set(style='whitegrid')
%matplotlib inline

print("[OK] Bibliotecas importadas com sucesso!")

In [ ]:
# Download recursos NLTK
recursos = ['punkt', 'stopwords', 'wordnet', 'punkt_tab', 'rslp']
for recurso in recursos:
    nltk.download(recurso, quiet=True)

print("[OK] Recursos NLTK baixados!")

In [ ]:
# Configurar caminhos
DATA_RAW = r'c:\Users\pedro\Downloads\Senti-Pred\data\raw\Test.csv'
DATA_PROCESSED = r'c:\Users\pedro\Downloads\Senti-Pred\data\processed\processed_data.csv'
MODEL_PATH = r'c:\Users\pedro\Downloads\Senti-Pred\src\models\sentiment_model.pkl'

# Criar diretorios se nao existirem
os.makedirs(os.path.dirname(DATA_PROCESSED), exist_ok=True)
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
os.makedirs('../reports/visualizacoes', exist_ok=True)

print("[OK] Estrutura de diretorios configurada!")

---
## 2. Analise Exploratoria de Dados (EDA) <a id='eda'></a>

In [ ]:
# Carregar dados
print("Carregando dados...")
df = pd.read_csv(DATA_RAW)
print(f"[OK] Dados carregados: {df.shape[0]} registros e {df.shape[1]} colunas")

# Exibir primeiras linhas
print("\nPrimeiras linhas do dataset:")
display(df.head())

# Informações básicas
print("\nInformações do dataset:")
df.info()

# Estatísticas descritivas
print("\nEstatísticas descritivas:")
display(df.describe(include='all'))

# Verificar valores nulos
print("\nValores nulos por coluna:")
display(df.isnull().sum())

In [ ]:
# Distribuição de classes (assumindo que a coluna de sentimento é 'sentiment')
if 'sentiment' in df.columns:
    print("\nDistribuição de classes:")
    sentiment_counts = df['sentiment'].value_counts()
    display(sentiment_counts)
    
    # Visualizar distribuição
    plt.figure(figsize=(10, 6))
    sns.countplot(x='sentiment', data=df)
    plt.title('Distribuição de Sentimentos')
    plt.xlabel('Sentimento')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    # Se não houver coluna 'sentiment', assumir que a última coluna é a target
    target_col = df.columns[-1]
    print(f"\nDistribuição de classes (coluna {target_col}):")
    target_counts = df[target_col].value_counts()
    display(target_counts)
    
    # Visualizar distribuição
    plt.figure(figsize=(10, 6))
    sns.countplot(x=target_col, data=df)
    plt.title(f'Distribuição de {target_col}')
    plt.xlabel(target_col)
    plt.ylabel('Contagem')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Análise de comprimento de texto
print("\nAnálise de comprimento de texto:")
# Identificar a coluna de texto (assumindo que é a primeira coluna ou se chama 'text')
text_col = 'text' if 'text' in df.columns else df.columns[0]

df['text_length'] = df[text_col].apply(lambda x: len(str(x).split()))
print(f"Comprimento médio: {df['text_length'].mean():.2f} palavras")
print(f"Comprimento mínimo: {df['text_length'].min()} palavras")
print(f"Comprimento máximo: {df['text_length'].max()} palavras")

# Visualizar distribuição de comprimento
plt.figure(figsize=(12, 6))
sns.histplot(df['text_length'], bins=50, kde=True)
plt.title('Distribuição de Comprimento de Texto')
plt.xlabel('Número de Palavras')
plt.ylabel('Frequência')
plt.axvline(df['text_length'].mean(), color='red', linestyle='--', label=f'Média: {df["text_length"].mean():.2f}')
plt.legend()
plt.tight_layout()
plt.savefig('../reports/visualizacoes/text_lenght.png')
plt.show()

In [ ]:
# Palavras mais comuns
print("\nPalavras mais comuns:")
all_words = ' '.join(df[text_col].astype(str)).lower().split()
word_counts = pd.Series(all_words).value_counts()
print(word_counts.head(20))

# Visualizar palavras mais comuns
plt.figure(figsize=(12, 6))
word_counts[:20].plot(kind='bar')
plt.title('20 Palavras Mais Frequentes')
plt.xlabel('Palavra')
plt.ylabel('Frequência')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../reports/visualizacoes/number_words.png')
plt.show()

print("[OK] Análise exploratória concluída!")

---
## 3. Pre-processamento <a id='preprocessing'></a>

In [ ]:
# Definir funções de pré-processamento
def clean_text(text):
    """
    Limpa o texto removendo URLs, menções, pontuações e números
    """
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_stopwords(text):
    """
    Remove stopwords do texto
    """
    if not isinstance(text, str):
        return ''
    stop_words = set(stopwords.words('portuguese'))
    tokens = word_tokenize(text, language='portuguese')
    filtered = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered)

def lemmatize_text(text):
    """
    Lematiza o texto
    """
    if not isinstance(text, str):
        return ''
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text, language='portuguese')
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

print("[OK] Funções de pré-processamento definidas!")

In [ ]:
# Aplicar pré-processamento
print("Aplicando pré-processamento...")

# Identificar colunas
text_col = 'text' if 'text' in df.columns else df.columns[0]
target_col = 'sentiment' if 'sentiment' in df.columns else df.columns[-1]

# Criar cópia para processamento
df_processed = df.copy()

# Aplicar etapas de pré-processamento
print("1. Limpando textos...")
df_processed['text_clean'] = df_processed[text_col].apply(clean_text)

print("2. Removendo stopwords...")
df_processed['text_no_stop'] = df_processed['text_clean'].apply(remove_stopwords)

print("3. Lematizando textos...")
df_processed['text_lemmatized'] = df_processed['text_no_stop'].apply(lemmatize_text)

# Exibir exemplos
print("\nExemplos de textos processados:")
examples = pd.DataFrame({
    'Original': df_processed[text_col].head(3),
    'Limpo': df_processed['text_clean'].head(3),
    'Sem Stopwords': df_processed['text_no_stop'].head(3),
    'Lematizado': df_processed['text_lemmatized'].head(3)
})
display(examples)

# Salvar dados processados
df_processed.to_csv(DATA_PROCESSED, index=False)
print(f"[OK] Dados processados salvos em: {DATA_PROCESSED}")

---
## 4. Modelagem <a id='modeling'></a>

In [ ]:
# Preparar dados para modelagem
print("Preparando dados para modelagem...")

# Definir features e target
X = df_processed['text_lemmatized']
y = df_processed[target_col]

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Dados de treino: {X_train.shape[0]} amostras")
print(f"Dados de teste: {X_test.shape[0]} amostras")

# Verificar distribuição nos conjuntos
print("\nDistribuição de classes:")
print(f"Treino:\n{y_train.value_counts(normalize=True)}")
print(f"\nTeste:\n{y_test.value_counts(normalize=True)}")

In [ ]:
# Definir e treinar modelos
print("\n" + "="*60)
print("TREINAMENTO DE MODELOS")
print("="*60)

# Definir modelos a serem testados
models_config = {
    'Regressão Logística': LogisticRegression(max_iter=1000, random_state=42),
    'Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Dicionário para armazenar resultados
models = {}

# Treinar e avaliar cada modelo
for name, model in models_config.items():
    print(f"\nTreinando {name}...")
    
    # Criar pipeline com vetorização TF-IDF
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ('model', model)
    ])
    
    # Treinar
    pipeline.fit(X_train, y_train)
    
    # Predizer
    y_pred = pipeline.predict(X_test)
    
    # Avaliar
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia: {accuracy:.4f}")
    
    # Salvar modelo e predições
    models[name] = (pipeline, y_pred, accuracy)
    
    # Exibir matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=pipeline.classes_, 
                yticklabels=pipeline.classes_)
    plt.title(f'Matriz de Confusão - {name}')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.tight_layout()
    plt.savefig(f'../reports/visualizacoes/{name.lower().replace(" ", "-")}_matriz.png')
    plt.show()
    
    # Exibir relatório de classificação
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

# Identificar melhor modelo
best_model_name = max(models, key=lambda k: models[k][2])
best_model = models[best_model_name][0]
best_accuracy = models[best_model_name][2]

print("\n" + "="*60)
print(f"MELHOR MODELO: {best_model_name} (Acurácia: {best_accuracy:.4f})")
print("="*60)

In [ ]:
# Comparar modelos
print("\n" + "="*60)
print("COMPARAÇÃO DE MODELOS")
print("="*60)

# Extrair acurácias
model_names = list(models.keys())
accuracies = [models[name][2] for name in model_names]

# Visualizar comparação
plt.figure(figsize=(12, 6))
bars = plt.bar(model_names, accuracies, color=['blue', 'green', 'purple'])
plt.title('Comparação de Acurácia entre Modelos')
plt.xlabel('Modelo')
plt.ylabel('Acurácia')
plt.ylim(0, 1)

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/visualizacoes/models_comparasion.png')
plt.show()

# Salvar melhor modelo
print(f"\nSalvando melhor modelo ({best_model_name})...")
joblib.dump(best_model, MODEL_PATH)

# Salvar informações do modelo
info_path = os.path.join(os.path.dirname(MODEL_PATH), 'model_info.txt')
with open(info_path, 'w') as f:
    f.write(f"Modelo: {best_model_name}\n")
    f.write(f"Acurácia: {best_accuracy:.4f}\n")
    f.write(f"Data de treinamento: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}\n")
    f.write(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras\n")
    f.write(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras\n")
    
    # Adicionar relatório de classificação
    f.write("\nRelatório de Classificação:\n")
    f.write(classification_report(y_test, models[best_model_name][1]))

print(f"[OK] Modelo salvo em: {MODEL_PATH}")
print(f"[OK] Informações salvas em: {info_path}")

---
## 5. Avaliacao Detalhada <a id='evaluation'></a>

In [ ]:
# Analise de erros
print("\n" + "="*60)
print("ANALISE DE ERROS")
print("="*60)

y_pred_best = models[best_model_name][1]

# Identificar erros
errors = y_test != y_pred_best
error_indices = np.where(errors)[0]

error_df = pd.DataFrame({
    'Texto': X_test.iloc[error_indices].values,
    'Real': y_test.iloc[error_indices].values,
    'Predito': y_pred_best[error_indices]
})

print(f"\nTotal de erros: {len(error_indices)} ({len(error_indices)/len(y_test)*100:.2f}%)")
print(f"Total de acertos: {len(y_test) - len(error_indices)} ({(1 - len(error_indices)/len(y_test))*100:.2f}%)")

if len(error_indices) > 0:
    print("\nExemplos de erros:")
    display(error_df.head(10))

In [ ]:
# Avaliacao por comprimento de texto
print("\n" + "="*60)
print("AVALIACAO POR COMPRIMENTO")
print("="*60)

# Calcular comprimento dos textos de teste
text_lengths = X_test.apply(lambda x: len(str(x).split()))

# Categorizar
bins = [0, 5, 10, 20, float('inf')]
labels = ['Muito Curto', 'Curto', 'Medio', 'Longo']
length_categories = pd.cut(text_lengths, bins=bins, labels=labels)

# Calcular acuracia por categoria
accuracy_by_length = {}
for category in labels:
    mask = length_categories == category
    if mask.sum() > 0:
        acc = accuracy_score(y_test[mask], y_pred_best[mask])
        accuracy_by_length[category] = acc
        print(f"   {category:15s}: {acc:.4f} ({mask.sum()} textos)")

# Visualizar
if accuracy_by_length:
    plt.figure(figsize=(10, 6))
    plt.bar(accuracy_by_length.keys(), accuracy_by_length.values(), color='purple')
    plt.title('Acuracia por Comprimento de Texto')
    plt.xlabel('Categoria')
    plt.ylabel('Acuracia')
    plt.ylim(0, 1)
    for i, (cat, acc) in enumerate(accuracy_by_length.items()):
        plt.text(i, acc + 0.02, f"{acc:.3f}", ha='center', fontweight='bold')
    plt.tight_layout()
    plt.savefig('../reports/visualizacoes/precision.png')
    plt.show()

In [ ]:
# Curvas ROC (se aplicavel)
if len(best_model.classes_) == 2 and hasattr(best_model, 'predict_proba'):
    print("\n" + "="*60)
    print("CURVAS ROC E PRECISION-RECALL")
    print("="*60)
    
    y_prob = best_model.predict_proba(X_test)[:, 1]
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Curva ROC
    fpr, tpr, _ = roc_curve(y_test, y_prob, pos_label=best_model.classes_[1])
    roc_auc = auc(fpr, tpr)
    
    axes[0].plot(fpr, tpr, color='darkorange', lw=2, 
                 label=f'ROC (AUC = {roc_auc:.2f})')
    axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0].set_xlim([0.0, 1.0])
    axes[0].set_ylim([0.0, 1.05])
    axes[0].set_xlabel('Taxa de Falsos Positivos')
    axes[0].set_ylabel('Taxa de Verdadeiros Positivos')
    axes[0].set_title('Curva ROC')
    axes[0].legend(loc="lower right")
    axes[0].grid(True, alpha=0.3)
    
    # Curva Precision-Recall
    precision, recall, _ = precision_recall_curve(y_test, y_prob, pos_label=best_model.classes_[1])
    
    axes[1].plot(recall, precision, color='green', lw=2)
    axes[1].set_xlabel('Recall')
    axes[1].set_ylabel('Precision')
    axes[1].set_title('Curva Precision-Recall')
    axes[1].set_ylim([0.0, 1.05])
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n[OK] AUC-ROC: {roc_auc:.4f}")

In [ ]:
# Funcao de predicao
def predict_sentiment(text, model=best_model):
    """
    Prediz sentimento de um texto
    
    Args:
        text (str): Texto para analise
        model: Modelo treinado
    
    Returns:
        dict: Resultado com sentimento e confianca
    """
    # Pre-processar
    text_clean = clean_text(text)
    text_no_stop = remove_stopwords(text_clean)
    text_lem = lemmatize_text(text_no_stop)
    
    # Predizer
    sentiment = model.predict([text_lem])[0]
    
    # Confianca
    if hasattr(model, 'predict_proba'):
        probs = model.predict_proba([text_lem])[0]
        confidence = max(probs)
    else:
        confidence = 1.0
    
    return {
        'texto_original': text,
        'texto_processado': text_lem,
        'sentimento': sentiment,
        'confianca': confidence
    }

print("[OK] Funcao de predicao criada!")

In [ ]:
# Testar com exemplos
print("\n" + "="*60)
print("TESTES COM EXEMPLOS")
print("="*60)

test_examples = [
    "Este produto e excelente! Recomendo muito!",
    "Pessima qualidade. Nao comprem.",
    "E okay, nada de especial.",
    "Adorei! Muito bom mesmo!",
    "Terrivel. Pior compra da minha vida."
]

for text in test_examples:
    result = predict_sentiment(text)
    
    print(f"\nTexto: {result['texto_original']}")
    print(f"   Sentimento: {result['sentimento'].upper()}")
    print(f"   Confianca: {result['confianca']:.2%}")

---
## 6. Deploy com Docker <a id='deploy'></a>

### Estrutura de arquivos para deploy:

```
Senti-Pred/
├── app.py                 # API Flask
├── requirements.txt       # Dependencias
├── Dockerfile            # Configuracao Docker
├── docker-compose.yml    # Orquestracao
└── src/
    └── models/
        └── sentiment_model.pkl
```

In [ ]:
# Criar arquivo app.py (API Flask)
app_code = '''
from flask import Flask, request, jsonify
import joblib
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download recursos NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet=True)

app = Flask(__name__)

# Carregar modelo
model = joblib.load('src/models/sentiment_model.pkl')

# Funcoes de pre-processamento
def clean_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'http\\S+|www\\S+|https\\S+', '', text)
    text = re.sub(r'@\\w+|#\\w+', '', text)
    text = re.sub(r'[^\\w\\s]', '', text)
    text = re.sub(r'\\d+', '', text)
    text = re.sub(r'\\s+', ' ', text).strip()
    return text

def remove_stopwords(text):
    if not isinstance(text, str):
        return ''
    stop_words = set(stopwords.words('portuguese'))
    tokens = word_tokenize(text, language='portuguese')
    filtered = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered)

def lemmatize_text(text):
    if not isinstance(text, str):
        return ''
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text, language='portuguese')
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

def preprocess(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text

@app.route('/')
def home():
    return jsonify({
        'message': 'Senti-Pred API',
        'version': '1.0',
        'endpoints': {
            '/predict': 'POST - Analise de sentimento',
            '/health': 'GET - Status da API'
        }
    })

@app.route('/health')
def health():
    return jsonify({'status': 'healthy'})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        
        if not data or 'text' not in data:
            return jsonify({'error': 'Texto nao fornecido'}), 400
        
        text = data['text']
        
        # Pre-processar
        text_processed = preprocess(text)
        
        # Predizer
        sentiment = model.predict([text_processed])[0]
        
        # Confianca
        if hasattr(model, 'predict_proba'):
            probs = model.predict_proba([text_processed])[0]
            confidence = float(max(probs))
        else:
            confidence = 1.0
        
        return jsonify({
            'text': text,
            'sentiment': sentiment,
            'confidence': confidence
        })
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)
'''

# Salvar app.py
app_path = r'c:\Users\pedro\Downloads\Senti-Pred\app.py'
with open(app_path, 'w', encoding='utf-8') as f:
    f.write(app_code)

print(f"[OK] app.py criado em: {app_path}")

In [ ]:
# Criar requirements.txt
requirements = '''
flask==3.0.0
scikit-learn==1.3.0
pandas==2.1.0
numpy==1.25.0
joblib==1.3.2
nltk==3.8.1
gunicorn==21.2.0
'''

req_path = r'c:\Users\pedro\Downloads\Senti-Pred\requirements.txt'
with open(req_path, 'w', encoding='utf-8') as f:
    f.write(requirements.strip())

print(f"[OK] requirements.txt criado em: {req_path}")

In [ ]:
# Criar Dockerfile
dockerfile = '''
FROM python:3.11-slim

WORKDIR /app

# Copiar arquivos
COPY requirements.txt .
COPY app.py .
COPY src/ ./src/

# Instalar dependencias
RUN pip install --no-cache-dir -r requirements.txt

# Download recursos NLTK
RUN python -c "import nltk; nltk.download('punkt'); nltk.download('stopwords'); nltk.download('wordnet'); nltk.download('punkt_tab')"

# Expor porta
EXPOSE 5000

# Comando de execucao
CMD ["gunicorn", "--bind", "0.0.0.0:5000", "--workers", "4", "app:app"]
'''

dockerfile_path = r'c:\Users\pedro\Downloads\Senti-Pred\Dockerfile'
with open(dockerfile_path, 'w', encoding='utf-8') as f:
    f.write(dockerfile.strip())

print(f"[OK] Dockerfile criado em: {dockerfile_path}")

In [ ]:
# Criar docker-compose.yml
docker_compose = '''
version: '3.8'

services:
  senti-pred-api:
    build: .
    container_name: senti-pred
    ports:
      - "5000:5000"
    environment:
      - FLASK_ENV=production
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:5000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
'''

compose_path = r'c:\Users\pedro\Downloads\Senti-Pred\docker-compose.yml'
with open(compose_path, 'w', encoding='utf-8') as f:
    f.write(docker_compose.strip())

print(f"[OK] docker-compose.yml criado em: {compose_path}")

### Comandos para Deploy:

```bash
# 1. Navegar ate o diretorio do projeto
cd c:\Users\pedro\Downloads\Senti-Pred

# 2. Build e iniciar containers
docker-compose up --build -d

# 3. Verificar status
docker-compose ps

# 4. Ver logs
docker-compose logs -f

# 5. Testar API
curl -X POST http://localhost:5000/predict \
  -H "Content-Type: application/json" \
  -d '{"text": "Este produto e excelente!"}'

# 6. Parar containers
docker-compose down
```

In [ ]:
# Criar script de teste da API
test_script = '''
import requests
import json

API_URL = "http://localhost:5000"

def test_health():
    """Testa endpoint de health"""
    response = requests.get(f"{API_URL}/health")
    print(f"Health Check: {response.json()}")

def test_predict(text):
    """Testa predicao de sentimento"""
    response = requests.post(
        f"{API_URL}/predict",
        json={"text": text}
    )
    result = response.json()
    print(f"\\nTexto: {text}")
    print(f"Sentimento: {result.get('sentiment')}")
    print(f"Confianca: {result.get('confidence', 0):.2%}")

if __name__ == "__main__":
    # Testar health
    test_health()
    
    # Testar predicoes
    examples = [
        "Este produto e excelente! Recomendo!",
        "Pessima qualidade. Nao comprem.",
        "E okay, nada demais."
    ]
    
    for text in examples:
        test_predict(text)
'''

test_path = r'c:\Users\pedro\Downloads\Senti-Pred\test_api.py'
with open(test_path, 'w', encoding='utf-8') as f:
    f.write(test_script.strip())

print(f"[OK] test_api.py criado em: {test_path}")

### Instrucoes finais de deploy:

1. **Certifique-se que o Docker esta instalado e rodando**

2. **Execute no terminal:**
   ```bash
   docker-compose up --build -d
   ```

3. **API estara disponivel em:** `http://localhost:5000`

4. **Para testar a API:**
   ```bash
   python test_api.py
   ```

5. **Para parar os containers:**
   ```bash
   docker-compose down
   ```

### Exemplo de uso da API:

```python
import requests

# Analisar sentimento
response = requests.post(
    "http://localhost:5000/predict",
    json={"text": "Este produto é excelente!"}
)

# Exibir resultado
print(response.json())
```

### Conclusão

Neste notebook, implementamos um pipeline completo de análise de sentimentos, desde o pré-processamento dos dados até o deploy da solução em um container Docker. O modelo treinado pode ser utilizado para classificar textos em português de acordo com o sentimento expresso.

O pipeline inclui:
1. Análise exploratória dos dados
2. Pré-processamento de texto
3. Treinamento e comparação de modelos
4. Avaliação detalhada do melhor modelo
5. Deploy da solução como API REST

Para melhorias futuras, podemos considerar:
- Utilizar modelos mais avançados como BERT ou transformers
- Implementar técnicas de data augmentation
- Adicionar mais métricas de avaliação
- Implementar monitoramento da API em produção